In [0]:
import requests
import pandas as pd
import base64
import time
from pyspark.sql import SparkSession

# --- 1. CONFIGURARE UNITY CATALOG ---
CATALOG_NAME = "spotify_etl"
BRONZE_SCHEMA = "bronze"

spark = SparkSession.builder.getOrCreate()
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{BRONZE_SCHEMA}")

print(f"Folosind Catalog: {CATALOG_NAME}, Schema: {BRONZE_SCHEMA}")

# --- 2. CONFIGURARE SECRETE ȘI API ---
# Preluăm secretele necesare pentru reîmprospătare
try:
    # Atenție la numele scope-ului și cheilor, asigurați-vă că se potrivesc
    CLIENT_ID = dbutils.secrets.get(scope="spotify_secrets", key="user") # Ați folosit 'user', de obicei este 'client-id'
    CLIENT_SECRET = dbutils.secrets.get(scope="spotify_secrets", key="client_secret")
    REFRESH_TOKEN = dbutils.secrets.get(scope="spotify_secrets", key="refresh-token")
except Exception as e:
    print("EROARE: Nu s-au putut citi secretele (user, client_secret, refresh-token) din 'spotify_secrets'.")
    print("Asigură-te că ai urmat pașii de setup pentru a crea secretele.")
    raise e

# URL-ul de autentificare Spotify
TOKEN_URL = "https://accounts.spotify.com/api/token"
BASE_URL = "https://api.spotify.com/v1"

# Variabilă globală pentru a stoca token-ul curent
CURRENT_ACCESS_TOKEN = None

print("Configurarea și secretele au fost încărcate.")

In [0]:
def refresh_access_token():
    """
    Folosește REFRESH_TOKEN (din secrete) pentru a obține un nou access_token.
    """
    global CURRENT_ACCESS_TOKEN
    print("Refreshing access token...")
    
    auth_str = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_b64 = base64.b64encode(auth_str.encode()).decode()
    
    auth_data = {
        "grant_type": "refresh_token",
        "refresh_token": REFRESH_TOKEN
    }
    auth_headers = {"Authorization": f"Basic {auth_b64}"}
    
    try:
        response = requests.post(TOKEN_URL, data=auth_data, headers=auth_headers)
        response.raise_for_status()
        token_data = response.json()
        new_token = token_data.get('access_token')
        
        if not new_token:
            raise Exception("Răspunsul nu a conținut un 'access_token'")
            
        CURRENT_ACCESS_TOKEN = new_token
        print("Access token successfully refreshed.")
        return True
        
    except Exception as e:
        print(f"EROARE CRITICĂ la reîmprospătarea token-ului: {e}")
        CURRENT_ACCESS_TOKEN = None
        return False

def make_api_call(url, params=None, retries=1):
    """
    Funcție wrapper care gestionează apelurile API,
    reîmprospătarea token-ului și reîncercările.
    """
    global CURRENT_ACCESS_TOKEN
    
    if not CURRENT_ACCESS_TOKEN:
        if not refresh_access_token():
            raise Exception("Nu s-a putut obține token-ul inițial.")

    headers = {"Authorization": f"Bearer {CURRENT_ACCESS_TOKEN}"}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        
        # Verifică dacă token-ul a expirat
        if response.status_code in [401, 403] and retries > 0:
            print(f"Token expirat ({response.status_code}). Se reîmprospătează...")
            if refresh_access_token():
                print("Se reîncearcă apelul API cu noul token...")
                return make_api_call(url, params=params, retries=0)
            else:
                response.raise_for_status()
        
        response.raise_for_status()
        return response.json()
    
    except requests.exceptions.HTTPError as e:
        print(f"Eroare API la apelul {url}: {e}")
        raise e

In [0]:
def chunk_list(data, size):
    """Împarte o listă în bucăți mai mici."""
    for i in range(0, len(data), size):
        yield data[i:i + size]

In [0]:
# ============================================================
# SPOTIFY BRONZE INGESTION - FETCH FUNCTIONS (OPTIMIZED)
# ============================================================

def get_play_history(unique_track_ids, unique_artist_ids):
    """Sursa A: Obține istoricul ascultărilor recente."""
    print("Fetching play history...")
    play_history_data = []
    url = f"{BASE_URL}/me/player/recently-played?limit=50"
    
    try:
        results = make_api_call(url)
        for item in results.get('items', []):
            if not item:
                continue

            track = item.get('track', {})
            artist = (track.get('artists') or [{}])[0]

            play_history_data.append({
                'played_at': item.get('played_at'),
                'track_id': track.get('id'),
                'track_name': track.get('name'),
                'artist_id': artist.get('id'),
                'artist_name': artist.get('name'),
                'album_id': track.get('album', {}).get('id'),
                'album_name': track.get('album', {}).get('name'),
                'context_type': (item.get('context') or {}).get('type'),
                'duration_ms': track.get('duration_ms')
            })

            if track.get('id'):
                unique_track_ids.add(track.get('id'))
            if artist.get('id'):
                unique_artist_ids.add(artist.get('id'))

    except Exception as e:
        print(f"⚠️ Error fetching play history: {e}")
    
    print(f"✅ Retrieved {len(play_history_data)} play history records.")
    return play_history_data


def get_playlists_and_tracks(unique_track_ids, unique_artist_ids):
    """Sursa B: Obține playlist-urile și piesele acestora."""
    print("Fetching playlists and their tracks...")
    playlists_data = []
    playlist_tracks_data = []
    
    next_url = f"{BASE_URL}/me/playlists?limit=50"
    
    try:
        while next_url:
            playlists = make_api_call(next_url)
            for playlist in playlists.get('items', []):
                playlist_id = playlist.get('id')
                if not playlist_id:
                    continue

                playlists_data.append({
                    'playlist_id': playlist_id,
                    'playlist_name': playlist.get('name'),
                    'owner_name': playlist.get('owner', {}).get('display_name'),
                    'followers': (playlist.get('followers') or {}).get('total'),
                    'total_tracks': (playlist.get('tracks') or {}).get('total'),
                    'description': playlist.get('description'),
                    'snapshot_id': playlist.get('snapshot_id')
                })

                # Fetch all tracks from the playlist
                tracks_url = f"{BASE_URL}/playlists/{playlist_id}/tracks"
                while tracks_url:
                    tracks = make_api_call(tracks_url)
                    for item in tracks.get('items', []):
                        track = item.get('track')
                        if not track or not track.get('id'):
                            continue

                        artist_ids = [a['id'] for a in (track.get('artists') or []) if a.get('id')]

                        playlist_tracks_data.append({
                            'playlist_id': playlist_id,
                            'track_id': track.get('id'),
                            'track_name': track.get('name'),
                            'artist_ids': artist_ids,
                            'album_id': track.get('album', {}).get('id'),
                            'added_at': item.get('added_at'),
                            'added_by': (item.get('added_by') or {}).get('id'),
                            'duration_ms': track.get('duration_ms'),
                            'popularity': track.get('popularity')
                        })

                        unique_track_ids.add(track.get('id'))
                        for art_id in artist_ids:
                            unique_artist_ids.add(art_id)

                    tracks_url = tracks.get('next')

            next_url = playlists.get('next')

    except Exception as e:
        print(f"⚠️ Error fetching playlists: {e}")

    print(f"✅ Retrieved {len(playlists_data)} playlists and {len(playlist_tracks_data)} playlist-track mappings.")
    return playlists_data, playlist_tracks_data


def get_full_metadata(unique_track_ids, unique_artist_ids):
    """Pasul 3: Îmbogățirea datelor (tracks, artists)."""
    print(f"Fetching metadata for {len(unique_track_ids)} tracks and {len(unique_artist_ids)} artists...")

    tracks_data, artists_data = [], []

    track_ids_list = list(filter(None, unique_track_ids))
    artist_ids_list = list(filter(None, unique_artist_ids))

    # 1️⃣ Fetch track metadata
    try:
        url = f"{BASE_URL}/tracks"
        for batch in chunk_list(track_ids_list, 50):
            params = {'ids': ','.join(batch)}
            tracks_results = make_api_call(url, params=params)
            for track in tracks_results.get('tracks', []):
                if not track:
                    continue
                tracks_data.append({
                    'track_id': track.get('id'),
                    'track_name': track.get('name'),
                    'album_id': track.get('album', {}).get('id'),
                    'album_name': track.get('album', {}).get('name'),
                    'artist_id': (track.get('artists') or [{}])[0].get('id'),
                    'artist_name': (track.get('artists') or [{}])[0].get('name'),
                    'duration_ms': track.get('duration_ms'),
                    'popularity': track.get('popularity'),
                    'explicit': track.get('explicit'),
                    'release_date': track.get('album', {}).get('release_date'),
                    'preview_url': track.get('preview_url')
                })
    except Exception as e:
        print(f"⚠️ Error fetching tracks metadata: {e}")

    # 2️⃣ Fetch artist metadata
    try:
        url = f"{BASE_URL}/artists"
        for batch in chunk_list(artist_ids_list, 50):
            params = {'ids': ','.join(batch)}
            artists_results = make_api_call(url, params=params)
            for artist in artists_results.get('artists', []):
                if not artist:
                    continue
                artists_data.append({
                    'artist_id': artist.get('id'),
                    'artist_name': artist.get('name'),
                    'genres': artist.get('genres'),
                    'followers': (artist.get('followers') or {}).get('total'),
                    'popularity': artist.get('popularity'),
                    'uri': artist.get('uri')
                })
    except Exception as e:
        print(f"⚠️ Error fetching artists metadata: {e}")


    print(f"✅ Retrieved: {len(tracks_data)} tracks, {len(artists_data)} artists.")
    return tracks_data, artists_data


In [0]:
def save_as_delta_table(all_data):
    """
    Salvează fiecare listă de dicționare ca o tabelă Delta
    în schema 'bronze' din Unity Catalog.
    """
    print(f"\n--- STARTING DATA LOAD (Saving to Delta Lake in {CATALOG_NAME}.{BRONZE_SCHEMA}) ---")
    
    for table_name, data_list in all_data.items():
        if not data_list:
            print(f"Skipping '{table_name}', no data found.")
            continue
        
        try:
            pd_df = pd.DataFrame(data_list)
            spark_df = spark.createDataFrame(pd_df)
            full_table_name = f"{CATALOG_NAME}.{BRONZE_SCHEMA}.{table_name}"
            
            print(f"Writing {spark_df.count()} rows to {full_table_name}...")
            spark_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(full_table_name)
            
            print(f"Successfully saved: {full_table_name}")
            
        except Exception as e:
            print(f"Error saving {table_name}: {e}")

In [0]:
print("--- STARTING SPOTIFY BRONZE PIPELINE ---")

unique_track_ids = set()
unique_artist_ids = set()

# --- 1. EXTRACT ---
print("\n--- PASUL 1: EXTRAGERE (API -> Python) ---")
play_history_table = get_play_history(unique_track_ids, unique_artist_ids)
playlists_table, playlist_tracks_table = get_playlists_and_tracks(unique_track_ids, unique_artist_ids)

# --- 2. ENRICH (API -> Python) ---
print("\n--- PASUL 2: ÎMBOGĂȚIRE (API -> Python) ---")
tracks_table, artists_table = get_full_metadata(unique_track_ids, unique_artist_ids)

print("\n--- REZUMAT EXTRAGERE ---")
print(f"Date bronze_play_history:   {len(play_history_table)} rânduri")
print("-----------------------------")
print(f"Date bronze_playlists:      {len(playlists_table)} rânduri")
print(f"Date bronze_playlist_tracks:{len(playlist_tracks_table)} rânduri")
print("-----------------------------")
print(f"Date bronze_tracks:         {len(tracks_table)} rânduri")
print(f"Date bronze_artists:        {len(artists_table)} rânduri")


# --- 3. LOAD (Python -> Delta) ---
print("\n--- PASUL 3: ÎNCĂRCARE (Python -> Delta) ---")
all_bronze_data = {
    "bronze_play_history": play_history_table,
    "bronze_playlists": playlists_table,
    "bronze_playlist_tracks": playlist_tracks_table,
    "bronze_tracks": tracks_table,
    "bronze_artists": artists_table,
}

save_as_delta_table(all_bronze_data)

print("\n--- BRONZE PIPELINE FINISHED ---")